# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846871265428                   -0.70    4.8         
  2   -7.852325404565       -2.26       -1.53    1.0   18.4ms
  3   -7.852615618138       -3.54       -2.56    1.5   19.6ms
  4   -7.852646000023       -4.52       -2.89    2.8   26.5ms
  5   -7.852646512727       -6.29       -3.20    1.0   18.7ms
  6   -7.852646680061       -6.78       -4.10    1.0   17.8ms
  7   -7.852646686647       -8.18       -5.27    2.0   22.2ms
  8   -7.852646686726      -10.10       -5.57    1.8   21.2ms
  9   -7.852646686728      -11.79       -5.66    1.5   19.5ms
 10   -7.852646686730      -11.73       -6.81    1.0   18.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846883046273                   -0.70           4.8         
  2   -7.852554095909       -2.25       -1.63   0.80    2.0    230ms
  3   -7.852639827042       -4.07       -2.71   0.80    1.0   23.9ms
  4   -7.852646445875       -5.18       -3.38   0.80    2.0   25.2ms
  5   -7.852646683085       -6.62       -4.23   0.80    1.8   19.4ms
  6   -7.852646686618       -8.45       -4.81   0.80    1.5   18.5ms
  7   -7.852646686719       -9.99       -5.79   0.80    1.2   16.9ms
  8   -7.852646686730      -10.98       -6.34   0.80    2.2   21.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.462226e+01     3.640882e+00
 * time: 0.04753398895263672
     1     1.044508e+00     1.651805e+00
 * time: 0.22678899765014648
     2    -1.648150e+00     2.072426e+00
 * time: 0.24460101127624512
     3    -3.897467e+00     1.683857e+00
 * time: 0.2702138423919678
     4    -5.197690e+00     1.501839e+00
 * time: 0.29592084884643555
     5    -6.927071e+00     8.693853e-01
 * time: 0.32149386405944824
     6    -7.012195e+00     1.152945e+00
 * time: 0.3392050266265869
     7    -7.614714e+00     9.466935e-01
 * time: 0.3569619655609131
     8    -7.679130e+00     1.548233e+00
 * time: 0.37477993965148926
     9    -7.685399e+00     1.420199e+00
 * time: 0.3925440311431885
    10    -7.715811e+00     9.548208e-01
 * time: 0.41834497451782227
    11    -7.718399e+00     2.255766e+00
 * time: 0.4360690116882324
    12    -7.736395e+00     1.660829e+00
 * time: 0.4538228511810303
    13    -7.766771e+00     5.296769e-01
 * time: 0.487

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846897029988                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645927950                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.51s
  3   -7.852646686730      -13.35       -7.26    119ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.2797363088481584e-7
|ρ_newton - ρ_scfv| = 1.9132727604232573e-7
|ρ_newton - ρ_dm|   = 2.236311926614616e-9
